# Devoir 3 - Classification images
## INF889G - Vision par ordinateur

### Romain Pajean (PAJR77270104) - Edgardo Cuellar Sanchez (CUEE68350007)

### Exercice 1:  Création d’un ensemble de données

Alors nos données sont des bonbons haribo
